In [ ]:
#Import what technolgies you need
import pandas as pd


In [ ]:
# Load the Datasets and Print how many rows and columns
food_df = pd.read_csv("Louisville_Metro_KY_Inspection_Results_Food_Service_Establishments_1200720516040038198.csv")
gun_df = pd.read_csv("Gun_Violence_Data_3431704539549171373.csv")

print("Food Inspection Dataset:")
print("Shape:", food_df.shape)

print("\nGun Violence Dataset:")
print("Shape:", gun_df.shape)